# RBF Calculation Speed-Up

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from krr import KRR, rbf_derivative
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import rbf_kernel

### Get Data

In [4]:
random_state = 0
num_points = 1000
x_data = np.arange(0, num_points)
y_data = np.sin(x_data)

# split into training and testing
train_prnt = 0.7
x_train, x_test, y_train, y_test = \
    train_test_split(x_data, y_data,
                     train_size=train_prnt,
                     random_state=random_state)

/Users/eman/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
# make a new axis D [N x D]
x_train, x_test = x_train[:, np.newaxis], x_test[:, np.newaxis]
y_train, y_test = y_train[:, np.newaxis], y_test[:, np.newaxis]

# remove the mean from y training
y_train = y_train - np.mean(y_train)

param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "gamma": np.logspace(-2, 2, 5)}

# initialize the kernel ridge regression model
KRR_model = GridSearchCV(KernelRidge(kernel='rbf'), 
                         cv=5, param_grid=param_grid)

# fit model to data
KRR_model.fit(x_train, y_train)

# predict using the krr model
y_pred = KRR_model.predict(x_test)

# RBF Derivative

In [33]:
def rbf_derivative(x_train, x_function, weights, 
                   kernel_mat, gamma=1.0):
    """This function calculates the rbf derivative
    Parameters
    ----------
    x_train : array, [N x D]
        The training data used to find the kernel model.

    x_function  : array, [M x D]
        The test points (or vector) to use.

    weights   : array, [N x D]
        The weights found from the kernel model
            y = K * weights

    kernel_mat: array, [N x M], default: None
        The rbf kernel matrix with the similarities between the test
        points and the training points.

    n_derivative : int, (default = 1) {1, 2}
        chooses which nth derivative to calculate

    gamma : float, default: None
        the parameter for the rbf_kernel matrix function

    Returns
    -------

    derivative : array, [M x D]
        returns the derivative with respect to training points used in
        the kernel model and the test points.

    Information
    -----------
    Author: Juan Emmanuel Johnson
    Email : jej2744@rit.edu
            juan.johnson@uv.es
    """

    # initialize rbf kernel
    derivative = np.zeros(np.shape(x_function))

    # consolidate the parameters
    theta = 2 * gamma
    
    # loop through dimensions
    for dim in np.arange(0, np.shape(x_function)[1]):

        # loop through the number of test points
        for iTest in np.arange(0, np.shape(x_function)[0]):

            # loop through the number of test points
            for iTrain in np.arange(0, np.shape(x_train)[0]):

                # calculate the derivative for the test points
                derivative[iTest, dim] += theta * weights[iTrain] * \
                                          (x_train[iTrain, dim] -
                                           x_function[iTest, dim]) * \
                                          kernel_mat[iTrain, iTest]


    return derivative

In [34]:
# extract necessary parameters
gamma = KRR_model.best_params_['gamma']
weights = KRR_model.best_estimator_.dual_coef_
kernel = rbf_kernel(x_train, gamma=gamma)

### Timing the Current Function

In [35]:
%timeit rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, gamma=gamma)

1.62 s ± 57.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%prun rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, gamma=gamma)

         610 function calls in 1.693 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.691    1.691    1.693    1.693 <ipython-input-33-2035d8fbd3f7>:1(rbf_derivative)
      302    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.arange}
      303    0.000    0.000    0.000    0.000 fromnumeric.py:1565(shape)
        1    0.000    0.000    1.693    1.693 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.zeros}
        1    0.000    0.000    1.693    1.693 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [38]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [39]:
%lprun -f rbf_derivative rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, gamma=gamma)

Timer unit: 1e-06 s

Total time: 2.06357 s
File: <ipython-input-33-2035d8fbd3f7>
Function: rbf_derivative at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def rbf_derivative(x_train, x_function, weights, 
     2                                                              kernel_mat, gamma=1.0):
     3                                               """This function calculates the rbf derivative
     4                                               Parameters
     5                                               ----------
     6                                               x_train : array, [N x D]
     7                                                   The training data used to find the kernel model.
     8                                           
     9                                               x_function  : array, [M x D]
    10                                                   The test points (or vector) 

In [40]:
from numba import double
from numba.decorators import jit, autojit

In [41]:
rbf_derivative_numba = autojit(rbf_derivative)

In [42]:
print('Numba w/ AutoJit:')
%timeit rbf_derivative_numba(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)

Numba w/ Jit:
1.6 s ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [44]:
%%cython

cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def rbf_derivative_cython(np.int64_t[:, :] x_train, 
                          np.int64_t[:, :] x_function, 
                          np.float64_t[:] weights, 
                          np.float64_t[:, :] kernel_mat, 
                          np.float64_t gamma):

    cdef int d_dimensions = x_function.shape[1]
    cdef int n_test = x_function.shape[0]
    cdef int n_train = x_train.shape[0]
    cdef int idim, iTest, iTrain
    # initialize rbf kernel
    cdef np.float64_t[:,:] derivative = np.zeros(np.shape(x_function))

    # consolidate the parameters
    cdef np.float64_t theta = gamma + gamma


    # loop through dimensions
    for idim in np.arange(0, d_dimensions):

        # loop through the number of test points
        for iTest in np.arange(0, n_test):

            # loop through the number of test points
            for iTrain in np.arange(0, n_train):

                # calculate the derivative for the test points
                derivative[iTest, idim] += theta * weights[iTrain] * \
                                          (x_train[iTrain, idim] -
                                           x_function[iTest, idim]) * \
                                          kernel_mat[iTrain, iTest]
    return derivative

In [46]:
print('Pure Python:')
%timeit rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, gamma=gamma)

print('Numba w/ AutoJit:')
%timeit rbf_derivative_numba(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)

print('Cython:')
%timeit rbf_derivative_cython(x_train, x_test, kernel_mat=kernel, weights=weights.squeeze(), gamma=gamma)

Pure Python:
1.64 s ± 62.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Numba w/ Jit:
1.75 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Cython:
15.4 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Making Numba Faster

In [48]:
@jit
def rbf_derivative_numba2(x_train, x_function, weights, 
                   kernel_mat, gamma=1.0):
    """This function calculates the rbf derivative
    Parameters
    ----------
    x_train : array, [N x D]
        The training data used to find the kernel model.

    x_function  : array, [M x D]
        The test points (or vector) to use.

    weights   : array, [N x D]
        The weights found from the kernel model
            y = K * weights

    kernel_mat: array, [N x M], default: None
        The rbf kernel matrix with the similarities between the test
        points and the training points.

    n_derivative : int, (default = 1) {1, 2}
        chooses which nth derivative to calculate

    gamma : float, default: None
        the parameter for the rbf_kernel matrix function

    Returns
    -------

    derivative : array, [M x D]
        returns the derivative with respect to training points used in
        the kernel model and the test points.

    Information
    -----------
    Author: Juan Emmanuel Johnson
    Email : jej2744@rit.edu
            juan.johnson@uv.es
    """

    # initialize rbf kernel
    derivative = np.zeros(np.shape(x_function))

    # consolidate the parameters
    theta = 2 * gamma
    
    # loop through dimensions
    for dim in np.arange(0, np.shape(x_function)[1]):

        # loop through the number of test points
        for iTest in np.arange(0, np.shape(x_function)[0]):

            # loop through the number of test points
            for iTrain in np.arange(0, np.shape(x_train)[0]):

                # calculate the derivative for the test points
                derivative[iTest, dim] += theta * weights[iTrain] * \
                                          (x_train[iTrain, dim] -
                                           x_function[iTest, dim]) * \
                                          kernel_mat[iTrain, iTest]


    return derivative

In [50]:
print('Numba w/ Jit:')
%timeit rbf_derivative_numba2(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)


Numba w/ Jit:
1.64 s ± 20.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
fast_numba = jit()(rbf_derivative)

print('Numba w/ Jit:')
%timeit fast_numba(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)

Numba w/ Jit:
1.62 s ± 58.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
rbf_derivative_numba = jit(void(double[:], double[:,:], double[:,:], double[:,:], double[:]))(rbf_derivative_numba)

AttributeError: 'CPUDispatcher' object has no attribute '__defaults__'

In [28]:
%timeit rbf_derivative_numba(x_train, x_test, kernel_mat=kernel, weights=weights, n_derivative=1, gamma=gamma)

UntypedAttributeError: Failed at nopython (nopython frontend)
Unknown attribute 'shape' of type Module(<module 'numpy' from '/Users/eman/anaconda3/lib/python3.6/site-packages/numpy/__init__.py'>)
File "<ipython-input-26-6afc3b98ed16>", line 42
[1] During: typing of get attribute at <ipython-input-26-6afc3b98ed16> (42)

13.9 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
print(x_train.shape, x_train.dtype)
print(x_test.shape, x_test.dtype)
print(kernel.shape, kernel.dtype)
print(weights.squeeze().shape, weights.squeeze().dtype)
print(gamma.shape, gamma.dtype, gamma.astype(np.float).dtype)

(700, 1) int64
(300, 1) int64
(700, 700) float64
(700,) float64
() float64 float64
